<a href="https://colab.research.google.com/github/King-prime-x/deep-learning/blob/main/Depression_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
!pip install evaluate

In [ ]:
from datasets import DatasetDict, Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
from transformers import DataCollatorWithPadding

In [ ]:
dataset_dict = load_dataset("shawhin/phishing-site-classification")
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 2100
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 450
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 450
    })
})


In [ ]:
id2label = {0 : 'safe' , 1 : 'not safe'}
label2id = {'safe' : 0 , 'not safe' : 1}

model_path = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels = 2, id2label = id2label, label2id = label2id)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
for name, param in model.base_model.named_parameters():
  param.requires_grad = False

for name, param in model.base_model.named_parameters():
  if 'pooler' in name:
    param.requires_grad = True
'''for name, param in model.base_model.named_parameters():
  if name != 'pooler':
    param.requires_grad = False '''

"for name, param in model.base_model.named_parameters():\n  if name != 'pooler':\n    param.requires_grad = False "

In [ ]:
def preprocess_function(examples):
  return tokenizer(examples["text"], truncation = True)

tokenized_data = dataset_dict.map(preprocess_function, batched = True)

data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

Map:   0%|          | 0/2100 [00:00<?, ? examples/s]

In [ ]:
accuracy = evaluate.load('accuracy')
auc_score = evaluate.load('roc_auc')

def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  probabilities = np.exp(predictions) / np.exp(predictions).sum(-1, keepdims = True)
  positive_class_probs = probabilities[:,1]
  auc = np.round(auc_score.compute(prediction_scores = positive_class_probs, references = labels)['roc_auc'],3)
  predicted_classes = np.argmax(predictions, axis = 1)
  acc = np.round(accuracy.compute(predictions = predicted_classes, references = labels)['accuracy'],3)

  return {"accuracy": acc, "AUC": auc}

In [ ]:
lr = 2e-4
batch_size = 8
num_epochs = 3

training_args = TrainingArguments(
    output_dir = 'bert-phising-classifier-teacher',
    learning_rate = lr,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = num_epochs,
    logging_strategy = "epoch",
    eval_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True,
    report_to = "none"
)

In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_data['train'],
    eval_dataset = tokenized_data['test'],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics
)

trainer.train()

<ipython-input-35-ef1750a05b67>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.502400,0.380467,0.820000,0.913000
2,0.405600,0.360128,0.847000,0.934000
3,0.368600,0.322465,0.856000,0.936000
4,0.346800,0.414956,0.831000,0.944000
5,0.351800,0.313914,0.862000,0.945000
6,0.352600,0.293538,0.862000,0.949000
7,0.321400,0.307751,0.860000,0.947000
8,0.309400,0.294713,0.867000,0.949000
9,0.323800,0.285014,0.876000,0.950000
10,0.309500,0.296724,0.867000,0.951000


TrainOutput(global_step=2630, training_loss=0.35918267833869266, metrics={'train_runtime': 5432.9198, 'train_samples_per_second': 3.865, 'train_steps_per_second': 0.484, 'total_flos': 706603239165360.0, 'train_loss': 0.35918267833869266, 'epoch': 10.0})